# Table Maintenance — 스냅샷 만료, 고아 파일 정리, Manifest 재작성

이 노트북에서는 Iceberg 테이블의 **유지보수 작업**을 실습합니다.

Iceberg는 Time Travel을 위해 과거 스냅샷과 데이터 파일을 보관하지만, 무한히 쌓이면 스토리지 비용과 메타데이터 크기가 증가합니다. 주기적으로 정리해야 합니다.

## 3가지 유지보수 작업

| 작업 | 대상 | 효과 |
|------|------|------|
| **Expire Snapshots** | 오래된 스냅샷 | 스냅샷 메타데이터 + 그 스냅샷만 참조하던 데이터 파일 삭제 |
| **Remove Orphan Files** | 어떤 스냅샷도 참조하지 않는 파일 | 실패한 작업의 잔해(고아 파일) 삭제 |
| **Rewrite Manifests** | manifest 파일 | 작은 manifest들을 병합하여 쿼리 계획 최적화 |

### 권장 실행 순서

```
1. Compaction (rewrite_data_files)  — 데이터 파일 정리
2. Expire Snapshots                — 오래된 스냅샷 제거
3. Remove Orphan Files             — 잔여 파일 정리
```

> 이 순서가 중요합니다: Compaction이 새 파일을 생성하므로, 이후에 Expire Snapshots로 이전 파일을 정리하고, 마지막에 Orphan Files로 잔여물을 청소합니다.

## 환경 설정

In [1]:
import sys
sys.path.append('..')

import time
import json
import glob as glob_mod
import os
from datetime import datetime, timedelta
from utils.spark_setup import create_spark_session
from utils.data_generator import generate_orders, to_spark_df
from utils.file_explorer import show_tree, snapshot_tree, diff_tree, count_files, total_size

In [2]:
spark = create_spark_session()

TABLE_NAME = "demo.lab.maintenance_orders"
TABLE_PATH = "/home/jovyan/data/warehouse/lab/maintenance_orders"

Spark + Iceberg 세션 준비 완료 (warehouse: file:///home/jovyan/data/warehouse)


---
## 준비: 여러 스냅샷이 쌓인 테이블 만들기

유지보수의 효과를 관찰하기 위해, 여러 번의 INSERT/UPDATE/DELETE로 다수의 스냅샷을 생성합니다.

In [3]:
spark.sql(f"DROP TABLE IF EXISTS {TABLE_NAME}")

spark.sql(f"""
CREATE TABLE {TABLE_NAME} (
    order_id BIGINT,
    customer_id BIGINT,
    product_name STRING,
    order_date DATE,
    amount DECIMAL(10,2),
    status STRING
) USING ICEBERG
PARTITIONED BY (months(order_date))
""")

# 여러 배치로 데이터 삽입 → 스냅샷 누적
for i in range(5):
    orders = generate_orders(num_records=50, seed=i, id_offset=i*50+1)
    df = to_spark_df(spark, orders)
    df.writeTo(TABLE_NAME).append()
    print(f"배치 {i+1}/5 완료 (50건 삽입)")

# UPDATE로 추가 스냅샷 생성
spark.sql(f"UPDATE {TABLE_NAME} SET status = 'shipped' WHERE status = 'pending'")
print("UPDATE 완료: pending → shipped")

# DELETE로 추가 스냅샷 생성
spark.sql(f"DELETE FROM {TABLE_NAME} WHERE status = 'cancelled'")
print("DELETE 완료: cancelled 삭제")

total_records = spark.sql(f"SELECT COUNT(*) FROM {TABLE_NAME}").collect()[0][0]
print(f"\n최종 레코드 수: {total_records}")

배치 1/5 완료 (50건 삽입)
배치 2/5 완료 (50건 삽입)
배치 3/5 완료 (50건 삽입)
배치 4/5 완료 (50건 삽입)
배치 5/5 완료 (50건 삽입)
UPDATE 완료: pending → shipped
DELETE 완료: cancelled 삭제

최종 레코드 수: 214


In [4]:
# 현재 스냅샷 히스토리 확인
print("=== 스냅샷 히스토리 ===")
spark.sql(f"""
SELECT snapshot_id, committed_at, operation, summary
FROM {TABLE_NAME}.snapshots
ORDER BY committed_at
""").show(truncate=False)

=== 스냅샷 히스토리 ===
+-------------------+-----------------------+---------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|snapshot_id        |committed_at           |operation|summary                                                                                                                                                                                                                                                                                                                                                                            |
+-------------------+-----------------------+---------+------------------------------------------------------------------------

In [5]:
# 현재 파일 상태
print("유지보수 전 파일 구조:")
print("=" * 60)
show_tree(TABLE_PATH, max_depth=3)

print(f"\nParquet 파일 수: {count_files(TABLE_PATH)}")
print(f"총 크기: {total_size(TABLE_PATH):,} bytes")

유지보수 전 파일 구조:
├── data/
│   ├── order_date_month=2024-01/
│   │   ├── 00000-11-1eac623a-b970-43b2-a96f-8018fd9e9666-00001.parquet  (2.3 KB)
│   │   ├── 00000-17-5ff81d03-99ad-42da-9845-0a3b5e715967-00001.parquet  (2.3 KB)
│   │   ├── 00000-23-c2c2d837-3fbc-4d54-86e9-f0bde8c8f770-00001.parquet  (2.2 KB)
│   │   ├── 00000-29-6e3eebf4-48bd-43ef-ba78-252f07e33b9f-00001.parquet  (2.2 KB)
│   │   ├── 00000-39-785dae90-87fb-42db-946f-808c9703754e-00001.parquet  (2.9 KB)
│   │   ├── 00000-42-cbe6e704-901a-41d0-8b6c-8529864ab4c3-00001.parquet  (2.8 KB)
│   │   └── 00000-5-c3442cda-6d72-48ec-92fa-9037a4e25244-00001.parquet  (2.2 KB)
│   ├── order_date_month=2024-02/
│   │   ├── 00000-11-1eac623a-b970-43b2-a96f-8018fd9e9666-00002.parquet  (2.1 KB)
│   │   ├── 00000-17-5ff81d03-99ad-42da-9845-0a3b5e715967-00002.parquet  (2.3 KB)
│   │   ├── 00000-23-c2c2d837-3fbc-4d54-86e9-f0bde8c8f770-00002.parquet  (2.2 KB)
│   │   ├── 00000-29-6e3eebf4-48bd-43ef-ba78-252f07e33b9f-00002.parquet  (2.3 KB)
│   │  

---
## 실험 1: Expire Snapshots — 오래된 스냅샷 제거

`expire_snapshots`는 지정 시간보다 오래된 스냅샷을 삭제합니다.

삭제된 스냅샷이 **유일하게 참조하던** 데이터 파일도 함께 삭제됩니다.

### 주요 옵션

| 옵션 | 설명 |
|------|------|
| `older_than` | 이 시간보다 오래된 스냅샷 삭제 |
| `retain_last` | 최소 N개 스냅샷 유지 (기본값: 1) |

> **주의**: 스냅샷을 삭제하면 해당 시점으로 **Time Travel이 불가능**해집니다. 운영 환경에서는 적절한 보존 기간을 설정하세요.

In [6]:
# 스냅샷 만료 전 상태 기록
before_snapshots = spark.sql(f"SELECT * FROM {TABLE_NAME}.snapshots").collect()
before = snapshot_tree(TABLE_PATH)

print(f"만료 전 스냅샷 수: {len(before_snapshots)}")
print(f"만료 전 Parquet 파일 수: {count_files(TABLE_PATH)}")

만료 전 스냅샷 수: 7
만료 전 Parquet 파일 수: 21


In [7]:
# Expire Snapshots — 실습을 위해 현재 스냅샷들 중 최근 1개만 유지
# retain_last만 지정하면 기본 older_than(보통 now - 5 days) 때문에 방금 만든 스냅샷은 만료되지 않을 수 있음
max_committed_at = spark.sql(f"SELECT MAX(committed_at) AS ts FROM {TABLE_NAME}.snapshots").collect()[0]["ts"]
expire_cutoff = (max_committed_at + timedelta(seconds=1)).strftime("%Y-%m-%d %H:%M:%S")

print(f"스냅샷 만료 기준(older_than): {expire_cutoff}")
spark.sql(f"""
CALL demo.system.expire_snapshots(
    table => '{TABLE_NAME}',
    older_than => TIMESTAMP '{expire_cutoff}',
    retain_last => 1
)
""").show(truncate=False)

스냅샷 만료 기준(older_than): 2026-02-16 00:28:30
+------------------------+-----------------------------------+-----------------------------------+----------------------------+----------------------------+------------------------------+
|deleted_data_files_count|deleted_position_delete_files_count|deleted_equality_delete_files_count|deleted_manifest_files_count|deleted_manifest_lists_count|deleted_statistics_files_count|
+------------------------+-----------------------------------+-----------------------------------+----------------------------+----------------------------+------------------------------+
|18                      |0                                  |0                                  |11                          |6                           |0                             |
+------------------------+-----------------------------------+-----------------------------------+----------------------------+----------------------------+------------------------------+



In [8]:
# 만료 후 상태 확인
after_snapshots = spark.sql(f"SELECT * FROM {TABLE_NAME}.snapshots").collect()
after = snapshot_tree(TABLE_PATH)

print(f"스냅샷 수: {len(before_snapshots)} → {len(after_snapshots)}")
print(f"Parquet 파일 수: {count_files(TABLE_PATH)}")

print("\n파일 변경 사항:")
diff_tree(before, after)

스냅샷 수: 7 → 1
Parquet 파일 수: 3

파일 변경 사항:

[+] 추가된 파일 (1개):
    + metadata/v9.metadata.json  (3.7 KB)

[-] 삭제된 파일 (35개):
    - data/order_date_month=2024-01/00000-11-1eac623a-b970-43b2-a96f-8018fd9e9666-00001.parquet
    - data/order_date_month=2024-01/00000-17-5ff81d03-99ad-42da-9845-0a3b5e715967-00001.parquet
    - data/order_date_month=2024-01/00000-23-c2c2d837-3fbc-4d54-86e9-f0bde8c8f770-00001.parquet
    - data/order_date_month=2024-01/00000-29-6e3eebf4-48bd-43ef-ba78-252f07e33b9f-00001.parquet
    - data/order_date_month=2024-01/00000-39-785dae90-87fb-42db-946f-808c9703754e-00001.parquet
    - data/order_date_month=2024-01/00000-5-c3442cda-6d72-48ec-92fa-9037a4e25244-00001.parquet
    - data/order_date_month=2024-02/00000-11-1eac623a-b970-43b2-a96f-8018fd9e9666-00002.parquet
    - data/order_date_month=2024-02/00000-17-5ff81d03-99ad-42da-9845-0a3b5e715967-00002.parquet
    - data/order_date_month=2024-02/00000-23-c2c2d837-3fbc-4d54-86e9-f0bde8c8f770-00002.parquet
    - data/order_d

In [9]:
# 남은 스냅샷 확인
print("=== 만료 후 스냅샷 ===")
spark.sql(f"""
SELECT snapshot_id, committed_at, operation
FROM {TABLE_NAME}.snapshots
ORDER BY committed_at
""").show(truncate=False)

=== 만료 후 스냅샷 ===
+-------------------+-----------------------+---------+
|snapshot_id        |committed_at           |operation|
+-------------------+-----------------------+---------+
|3126682812475079692|2026-02-16 00:28:29.684|overwrite|
+-------------------+-----------------------+---------+



### 관찰 포인트 — Expire Snapshots

- `older_than` + `retain_last => 1` 조합으로 **최근 1개 스냅샷만 남도록** 만료를 수행했습니다
- `retain_last`만 지정하면 기본 `older_than`(환경/설정에 따라 대개 최근 며칠) 때문에 방금 생성된 스냅샷은 남을 수 있습니다
- 만료된 스냅샷이 유일하게 참조하던 데이터 파일이 있다면 함께 삭제되어 **스토리지 절약**
- 이전 스냅샷으로의 Time Travel은 더 이상 불가능합니다
- 운영 환경에서는 `older_than`으로 시간 기반 만료를 설정하는 것이 일반적입니다:
  ```sql
  CALL system.expire_snapshots(
      table => 'my_table',
      older_than => TIMESTAMP '2024-01-01 00:00:00'
  )
  ```

---
## 실험 2: Remove Orphan Files — 고아 파일 정리

**고아 파일(Orphan Files)**은 어떤 스냅샷의 메타데이터에도 참조되지 않는 파일입니다.

### 고아 파일이 생기는 원인
- 실패한 Spark 작업이 중간에 파일을 생성한 경우
- 커밋 실패 후 롤백되었지만 물리 파일은 남은 경우
- 외부 도구가 잘못 파일을 생성한 경우

In [10]:
# 고아 파일 시뮬레이션: 테이블 디렉토리에 가짜 파일 생성
import os

data_dir = os.path.join(TABLE_PATH, "data")
if not os.path.exists(data_dir):
    # 파티션 디렉토리 중 하나를 사용
    for d in os.listdir(TABLE_PATH):
        full = os.path.join(TABLE_PATH, d)
        if os.path.isdir(full) and d != "metadata":
            data_dir = full
            break

orphan_path = os.path.join(data_dir, "orphan-fake-file.parquet")
with open(orphan_path, "wb") as f:
    f.write(b"fake orphan data " * 100)

print(f"고아 파일 시뮬레이션 생성: {orphan_path}")
print(f"파일 크기: {os.path.getsize(orphan_path)} bytes")

고아 파일 시뮬레이션 생성: /home/jovyan/data/warehouse/lab/maintenance_orders/data/orphan-fake-file.parquet
파일 크기: 1700 bytes


In [11]:
before = snapshot_tree(TABLE_PATH)

# Remove Orphan Files — dry_run으로 먼저 확인
# 주의: Iceberg는 기본적으로 3일 이내 파일은 고아로 판단하지 않음 (진행 중인 작업 보호)
# 실습에서는 older_than을 현재 시간 이후로 설정하여 강제 실행
from pyspark.sql.functions import current_timestamp, expr

future_ts = (datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")

print(f"고아 파일 탐색 (older_than: {future_ts})")
spark.sql(f"""
CALL demo.system.remove_orphan_files(
    table => '{TABLE_NAME}',
    older_than => TIMESTAMP '{future_ts}',
    dry_run => true
)
""").show(truncate=False)

고아 파일 탐색 (older_than: 2026-02-17 00:28:35)


IllegalArgumentException: Cannot remove orphan files with an interval less than 24 hours. Executing this procedure with a short interval may corrupt the table if other operations are happening at the same time. If you are absolutely confident that no concurrent operations will be affected by removing orphan files with such a short interval, you can use the Action API to remove orphan files with an arbitrary interval.

In [ ]:
# 실제로 고아 파일 삭제
spark.sql(f"""
CALL demo.system.remove_orphan_files(
    table => '{TABLE_NAME}',
    older_than => TIMESTAMP '{future_ts}'
)
""").show(truncate=False)

after = snapshot_tree(TABLE_PATH)

print("\n파일 변경 사항:")
diff_tree(before, after)

# 고아 파일이 삭제되었는지 확인
print(f"\n고아 파일 존재 여부: {os.path.exists(orphan_path)}")

### 관찰 포인트 — Remove Orphan Files

- `dry_run => true`로 **먼저 삭제 대상을 확인**한 후 실제 삭제를 수행했습니다
- 시뮬레이션으로 생성한 가짜 파일이 고아로 감지되어 삭제되었습니다
- 기본적으로 **3일 이내 파일은 보호**됩니다 (진행 중인 작업의 파일일 수 있으므로)
- 운영 환경에서는 `older_than`을 현재 시각에서 며칠 전으로 설정하세요:
  ```sql
  CALL system.remove_orphan_files(
      table => 'my_table',
      older_than => TIMESTAMP '2024-01-01 00:00:00'
  )
  ```

---
## 실험 3: Rewrite Manifests — Manifest 파일 병합

**Manifest 파일**은 데이터 파일의 위치, 크기, 통계 정보를 담고 있습니다.

INSERT가 반복되면 manifest 파일도 늘어나는데, 이를 병합하면:
- 쿼리 계획 시 읽어야 할 manifest 수 감소
- 파티션 프루닝 효율 향상

In [ ]:
# 현재 manifest 상태
print("=== Rewrite 전 Manifest 목록 ===")
manifests_before = spark.sql(f"SELECT * FROM {TABLE_NAME}.manifests")
manifests_before.show(truncate=False)
manifest_count_before = manifests_before.count()
print(f"Manifest 파일 수: {manifest_count_before}")

In [ ]:
# Rewrite Manifests 실행
spark.sql(f"""
CALL demo.system.rewrite_manifests(
    table => '{TABLE_NAME}'
)
""").show(truncate=False)

In [ ]:
# Rewrite 후 manifest 상태
print("=== Rewrite 후 Manifest 목록 ===")
manifests_after = spark.sql(f"SELECT * FROM {TABLE_NAME}.manifests")
manifests_after.show(truncate=False)
manifest_count_after = manifests_after.count()

print(f"\nManifest 파일 수: {manifest_count_before} → {manifest_count_after}")

### 관찰 포인트 — Rewrite Manifests

- 여러 개의 작은 manifest 파일이 **더 적은 수의 큰 manifest로 병합**되었습니다
- 쿼리 계획 시 읽어야 할 manifest 수가 줄어 **계획 수립 속도 향상**
- manifest 내 파티션 통계도 최적화되어 **프루닝 효율 개선**
- 이 작업은 데이터 파일 자체는 변경하지 않습니다 — **메타데이터만 재작성**

---
## 유지보수 자동화 가이드

### 권장 실행 주기

| 작업 | 주기 | 이유 |
|------|------|------|
| **Compaction** | 매시간 ~ 매일 | 스트리밍 수집 시 Small File Problem 방지 |
| **Expire Snapshots** | 매일 ~ 매주 | 스토리지 절약, 보존 기간에 따라 조정 |
| **Remove Orphan Files** | 매주 ~ 매월 | 빈도 낮아도 됨, dry_run 먼저 권장 |
| **Rewrite Manifests** | 매일 ~ 매주 | Compaction 후 실행하면 효과적 |

### Airflow DAG 예시 (의사 코드)

```python
# 매일 실행되는 유지보수 DAG
with DAG('iceberg_maintenance', schedule='@daily'):
    
    compact = SparkSubmitOperator(
        sql="CALL system.rewrite_data_files(table => 'my_table', strategy => 'sort')"
    )
    
    expire = SparkSubmitOperator(
        sql="CALL system.expire_snapshots(table => 'my_table', older_than => now() - INTERVAL 7 DAYS)"
    )
    
    orphan = SparkSubmitOperator(
        sql="CALL system.remove_orphan_files(table => 'my_table', older_than => now() - INTERVAL 3 DAYS)"
    )
    
    rewrite = SparkSubmitOperator(
        sql="CALL system.rewrite_manifests(table => 'my_table')"
    )
    
    compact >> expire >> orphan >> rewrite
```

### Time Travel과의 트레이드오프

```
스냅샷 보존 기간 ↑  →  스토리지 비용 ↑  +  Time Travel 범위 ↑
스냅샷 보존 기간 ↓  →  스토리지 비용 ↓  +  Time Travel 범위 ↓
```

운영 환경에서는 보통 **7일 보존**이 적절한 균형점입니다.

In [ ]:
spark.stop()
print("Spark 세션 종료")